# Part I. ETL Pipeline for Pre-Processing the Files

## Pre-processing the files into only one `csv` with the desired columns

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables to query with Apache Cassandra

Using the data file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory, that contains following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> file:<br>

<img src="images/image_event_datafile_new.jpg">

## Beginning of the Apache Cassandra code

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Creating a Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS u_de_project_02
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """
)
except Exception as e:
    print(e)

#### Setting the Keyspace

In [7]:
try:
    session.set_keyspace('u_de_project_02')
except Exception as e:
    print(e)

## Modelling the Apache Cassandra tables, based on the queries that should be run

### Following information should be queried from the tables
1. The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

2. The name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

3. Every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1: 
Returns the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The table `songs_by_session` has the columns `artist`, `song` and `length` that will be displayed in the results of the query and the columns `sessionId` and `itemInSession` that will be filtered. The primary key is composed of the partition key `sessionId` and clustering column `itemInSession`. Both are being used to filter the results

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session "
query += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        # Inserting data from csv and converting to correct data type
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

**Verifying that the table was created correctly and returns the expected data:**

In [15]:
query_01 = """
    SELECT
        artist,
        song,
        length
    FROM songs_by_session
    WHERE sessionId = 338
      AND itemInSession = 4
"""

try:
    rows = session.execute(query_01)
except Exception as e:
    print(e)
    
results_list = []
for row in rows:
    row_dict = {}
    row_dict['artist'] = row.artist
    row_dict['song'] = row.song
    row_dict['length'] = row.length
    results_list.append(row_dict)
    
df_results = pd.DataFrame(results_list)
df_results.head()

,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


### Answer
The answer to the query is:

Artist: Faithless | Song: Music Matters (Mark Knight Dub) | Length: 495.3 s


## Query 2:
Returns the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The table `songs_by_user` has the columns `artist`, `song`, `firstName` and `lastName` that will be displayed in the results of the query, the columns `userId` and `sessionId` that will be filtered and the column `itemInSession` to order the results. The primary key is composed of the partition keys `userId` and `sessionId` (both for filtering) and clustering column `itemInSession` for ordering.

In [17]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS songs_by_user "
query += "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        # Inserting data from csv and converting to correct data type
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

**Verifying that the table was created correctly and returns the expected data:**

In [21]:
query_02 = """
    SELECT
        artist,
        song,
        firstName,
        lastName
    FROM songs_by_user
    WHERE userId = 10
      AND sessionId = 182
    ORDER BY itemInSession
"""

try:
    rows = session.execute(query_02)
except Exception as e:
    print(e)

results_list = []
for row in rows:
    row_dict = {}
    row_dict['artist'] = row.artist
    row_dict['song'] = row.song
    row_dict['firstName'] = row.firstname
    row_dict['lastName'] = row.lastname
    results_list.append(row_dict)
    
df_results = pd.DataFrame(results_list)
df_results.head()

,artist,firstName,lastName,song
0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,Three Drives,Sylvie,Cruz,Greece 2000
2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


### Answer
The answer to the query is:

- Artist: Down To The Bone | Song: Keep On Keepin' On | Name of User (First and Last) Sylvie Cruz
- Artist: Three Drives | Song: Greece 2000 | Name of User (First and Last): Sylvie Cruz
- Artist: Sebastien Tellier | Song: Kilometer | Name of User (First and Last): Sylvie Cruz
- Artist: Lonnie Gordon | Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Name of User (First and Last): Sylvie Cruz

## Query 3: 
Returns every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The table `users_by_song` has the columns `firstName` and `lastName` that will be displayed in the results of the query, the column `song` that will be filtered and the column `userId` to uniquely identify each row along with `song`. The primary key is composed of the partition key `song` for filtering and clustering column `userId` to help uniquely identify each row.

In [23]:
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query += "(song text, firstName text, lastName text, userId int, PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song, firstName, lastName, userId) "
        query = query + "VALUES(%s, %s, %s, %s"
        # Inserting data from csv and converting to correct data type
        session.execute(query, (line[9], line[1], line[4], line[10]))

**Verifying that the table was created correctly and returns the expected data:**

In [25]:
query_03 = """
    SELECT
        firstName,
        lastName
    FROM users_by_song
    WHERE song = 'All Hands Against His Own'
    ORDER BY userId
"""

try:
    rows = session.execute(query_03)
except Exception as e:
    print(e)

results_list = []
for row in rows:
    row_dict = {}
    row_dict['firstName'] = row.firstname
    row_dict['lastName'] = row.lastname
    results_list.append(row_dict)
    
df_results = pd.DataFrame(results_list)
df_results.head()

,firstName,lastName
0,Sara,Johnson
1,Jacqueline,Lynch
2,Tegan,Levine


### Answer
The answer to the query is:

User Name (first and last): Sara Johnson
User Name (first and last): Jacqueline Lynch
User Name (first and last): Tegan Levine

### Dropping the tables

In [29]:
query = "DROP TABLE IF EXISTS songs_by_session"
try:
    drop = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS songs_by_user"
try:
    drop = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS users_by_song"
try:
    drop = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection

In [30]:
session.shutdown()
cluster.shutdown()